[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/akhil189/testp/blob/master/latest_answer.ipynb)


# User Session Analysis Case Study

In session analysis the key is to assess a user's behavior in the current organization workflow. To accomplish this challenge, machine learning models are trained on a sample of the dataset containing timeseries information about user activity during each session. To prepare the dataset in order to assess the behavior of the user during a day, aside from the overall session features such as logon and logoff timestamps, we need to identify features at each timestamp for each user to identify “normal” behavior.

In [ ]:
import pickle
from datetime import datetime, tzinfo, timezone, date, time
import numpy as np
import pandas as pd
import datetime

## Step 1 - Data Collection

For this exercise, please load the pickle from your local machine. 

In [ ]:
def load_pickle(entity):
    """
    @entity: str, entity name to be retrieved
    @does: generates a dataframe for the entity
    @return: a dataframe
    """
    entity_df = pickle.load(open(entity +'_challenge.pickle', 'rb'))
    
    for key in entity_df:
        entity_df[key].reset_index(inplace=True, drop=True)
    return entity_df

In [ ]:
df = load_pickle('session')
print(df.columns)

Index(['timestamp', 'domain_sid', 'gcp_logon_id', 'gcpDeviceId',
       'logon_session_type', 'logon_time', 'session_terminated',
       'termination_time'],
      dtype='object')


## Step 2 - Exploratory Data Analysis

At this step, we need to take out several steps to explore the data. The goal of this step is to get an understanding of the data structure and perform initial data preprocessing and cleaning.

In [ ]:
# Show the feature columns and first data entries
df.head()

,timestamp,domain_sid,gcp_logon_id,gcpDeviceId,logon_session_type,logon_time,session_terminated,termination_time
0,2020-12-20 19:43:39.450744+00:00,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t50F33B6F-9027-4C4B-86ED-10604E9082EA_6584685,t50F33B6F-9027-4C4B-86ED-10604E9082EA,LS_REMOTE_INTERACTIVE,2020-12-10 18:23:22.924000+00:00,False,2020-12-20 19:43:39.450744+00:00
1,2021-01-07 01:41:47.029254+00:00,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_231211946,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3,LS_REMOTE_INTERACTIVE,2021-01-07 01:40:40.708000+00:00,False,2021-01-07 13:01:06.033248+00:00
2,2020-12-21 02:47:43.532209+00:00,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3,LS_REMOTE_INTERACTIVE,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00
3,2020-12-21 19:28:25.682470+00:00,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3,LS_REMOTE_INTERACTIVE,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00
4,2020-12-21 21:15:41.034192+00:00,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3,LS_REMOTE_INTERACTIVE,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00


In [ ]:
# Get a summary on the data frame
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 997 entries, 0 to 1078
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   timestamp           997 non-null    object             
 1   domain_sid          997 non-null    object             
 2   gcp_logon_id        997 non-null    object             
 3   gcpDeviceId         997 non-null    object             
 4   logon_session_type  997 non-null    object             
 5   logon_time          997 non-null    datetime64[ns, UTC]
 6   session_terminated  997 non-null    bool               
 7   termination_time    997 non-null    datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](2), object(5)
memory usage: 63.3+ KB


Data in this example is cleaned and includes:
* timestamp
* domain_sid: this is considered a unique identifier of a user
* gcp_logon_id: this is considered a unique identifier of a session
* gcpDeviceId: this is considered a unique identifier of a device
* logon_session_type: Remote or Interactive sessions
* logon_time: timestamp of the session logon
* session_terminated: is session terminated?
* termination_time: termination time of a session if the session is terminated, otherwise, the last available timestamp of the session

In [ ]:
# evaluate how many users, devices, and for each user how many individual session record is available
unique_user_count = len(pd.unique(df['domain_sid']))
unique_device_count = len(pd.unique(df['gcpDeviceId']))

print(f'There are {unique_user_count} unique users and {unique_device_count} unique devices\n')

print(f'# of Individual login sessions for each user:\n')
df_user_sessions = df.groupby('domain_sid')['gcp_logon_id'].nunique()
# df.columns = ['user','# of individual sessions']
print(df_user_sessions)

There are 17 unique users and 371 unique devices

# of Individual login sessions for each user:

domain_sid
DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568-1709102071-1001     13
DESKTOP-D6AKRQI:S-1-5-21-3650031588-2315013739-4015488169-1001     32
DESKTOP-D6AKRQI:S-1-5-21-3650031588-2315013739-4015488169-1002      2
DESKTOP-G837B2C:S-1-5-21-773781223-2463708404-1040178729-1001     113
DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186-1228814399-1001      6
DMT:S-1-5-21-1480697204-747049651-2966119685-1001                   4
DMT:S-1-5-21-1480697204-747049651-2966119685-1002                  41
DMT:S-1-5-32-544                                                    1
MAURICIO-UYUNI:S-1-5-21-137712609-1835061822-942476125-1001         2
SHERMANBEASTVM:S-1-5-21-3858184157-4174427647-2357676267-1001       3
TAU:S-1-5-21-1075876963-3819156934-1349361978-1148                 96
TAU:S-1-5-21-1075876963-3819156934-1349361978-1149                  1
TAU:S-1-5-21-1075876963-3819156934-1349361978-1211  

In [ ]:
# Does the dataset include any missing values?
missing_vals = df.isnull().sum()
print(missing_vals)

print(f'\nNo, there are no missing values in the dataset')

timestamp             0
domain_sid            0
gcp_logon_id          0
gcpDeviceId           0
logon_session_type    0
logon_time            0
session_terminated    0
termination_time      0
dtype: int64

No, there are no missing values in the dataset


## Step 3 - Feature Generation

We are interested in creating features around a particular session for a user, for example the session duration for a particular user. Inorder to do so, we first need to make sure all instances of a session have the same logon_time and termination_time.

#### 1. check logon_time and termination_time for all instances of a session have the same logon_time and termination_time

In [ ]:
unique_session_count = df['gcp_logon_id'].nunique()
print(f'# of Unique Sessions = {unique_session_count}')

check_logon = df.groupby('gcp_logon_id')['logon_time'].nunique()
check_logon_df = pd.DataFrame(check_logon)
muliple_login_times_count = check_logon_df.loc[check_logon_df['logon_time'] > 1].shape[0]
print(f'# of Sessions with instances having multiple login timestamps = {muliple_login_times_count}')

check_termination = df.groupby('gcp_logon_id')['termination_time'].nunique()
check_termination_df = pd.DataFrame(check_termination)
multiple_terminations_count = check_termination_df.loc[check_termination_df['termination_time'] > 1].shape[0]

print(f'# of Sessions with instances having multiple termination timestamps = {multiple_terminations_count}')

# of Unique Sessions = 487
# of Sessions with instances having multiple login timestamps = 70
# of Sessions with instances having multiple termination timestamps = 74


#### 2. create a feature that calculates the overall session duration for a particular session

In [ ]:
print(f'orginal # of records - {df.shape[0]}')

orginal # of records - 997


In [ ]:
# Removing rows which has multiple session login times
df_clean = df.copy()
multiple_login_sessions_df = pd.DataFrame(check_logon_df.index[check_logon_df['logon_time'] > 1])
print(f'session ids which have more than one login timestamps - {multiple_login_sessions_df.shape[0]}')
df_clean = df_clean[~df_clean['gcp_logon_id'].isin(multiple_login_sessions_df['gcp_logon_id'])]
print(f'# of records after removing sessions with multiple login timestamps - {df_clean.shape[0]}')

session ids which have more than one login timestamps - 70
# of records after removing sessions with multiple login timestamps - 488


In [ ]:
# Removing rows which has multiple session termination times
multiple_termination_df = pd.DataFrame(check_termination_df.index[check_termination_df['termination_time'] > 1])
print(f'session ids which have more than one termination timestamps - {multiple_termination_df.shape[0]}')
df_clean = df_clean[~df_clean['gcp_logon_id'].isin(multiple_termination_df['gcp_logon_id'])]
print(f'# of records after removing sessions with multiple termination timestamps - {df_clean.shape[0]}')

session ids which have more than one termination timestamps - 74
# of records after removing sessions with multiple termination timestamps - 473


In [ ]:
# Rechecking if multiple logon times still exist
recheck_logon = df_clean.groupby('gcp_logon_id')['logon_time'].nunique()
recheck_logon_df = pd.DataFrame(recheck_logon)
muliple_login_times_count = recheck_logon_df.loc[recheck_logon_df['logon_time'] > 1].shape[0]
print(f'# of Sessions with instances having multiple login timestamps = {muliple_login_times_count}')

# of Sessions with instances having multiple login timestamps = 0


In [ ]:
# Rechecking if multiple termination times still exist
recheck_termination = df_clean.groupby('gcp_logon_id')['termination_time'].nunique()
recheck_termination_df = pd.DataFrame(recheck_termination)
multiple_terminations_count = recheck_termination_df.loc[recheck_termination_df['termination_time'] > 1].shape[0]
print(f'# of Sessions with instances having multiple termnination timestamps = {multiple_terminations_count}')

# of Sessions with instances having multiple termnination timestamps = 0


In [ ]:
# creating a new df
df_clean.columns

Index(['timestamp', 'domain_sid', 'gcp_logon_id', 'gcpDeviceId',
       'logon_session_type', 'logon_time', 'session_terminated',
       'termination_time'],
      dtype='object')

In [ ]:
users_df = df_clean.copy()
users_df = users_df.drop(['timestamp','gcpDeviceId', 'logon_session_type'], axis=1) 
users_df

,domain_sid,gcp_logon_id,logon_time,session_terminated,termination_time
0,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t50F33B6F-9027-4C4B-86ED-10604E9082EA_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-20 19:43:39.450744+00:00
1,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_231211946,2021-01-07 01:40:40.708000+00:00,False,2021-01-07 13:01:06.033248+00:00
2,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00
3,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00
4,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00
...,...,...,...,...,...
1037,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_317403,2020-11-05 16:26:09.857000+00:00,False,2020-11-05 18:11:27.919926+00:00
1038,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_238929,2020-11-05 16:00:56.040000+00:00,True,2020-11-05 16:23:38.421836+00:00
1039,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_302034,2020-11-05 18:10:53.857000+00:00,True,2020-11-05 18:14:33.960956+00:00
1040,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_1375164,2020-11-05 18:15:01.946000+00:00,False,2020-11-05 18:15:06.006260+00:00


In [ ]:
# creating a new feature for session

# df_clean['session_duration'] = df_clean['termination_time'] - df_clean['logon_time']
# print(f'# of records after removing sessions with instances having multiple logon and termination timestamps - {df_clean.shape[0]}')
users_df['session_duration'] = users_df['termination_time'] - users_df['logon_time']
users_df

,domain_sid,gcp_logon_id,logon_time,session_terminated,termination_time,session_duration
0,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t50F33B6F-9027-4C4B-86ED-10604E9082EA_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-20 19:43:39.450744+00:00,10 days 01:20:16.526744
1,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_231211946,2021-01-07 01:40:40.708000+00:00,False,2021-01-07 13:01:06.033248+00:00,0 days 11:20:25.325248
2,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00,11 days 10:21:36.879874
3,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00,11 days 10:21:36.879874
4,TAU:S-1-5-21-1075876963-3819156934-1349361978-...,t77025030-FF4B-4C0B-A16B-B0E131B8D4C3_6584685,2020-12-10 18:23:22.924000+00:00,False,2020-12-22 04:44:59.803874+00:00,11 days 10:21:36.879874
...,...,...,...,...,...,...
1037,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_317403,2020-11-05 16:26:09.857000+00:00,False,2020-11-05 18:11:27.919926+00:00,0 days 01:45:18.062926
1038,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_238929,2020-11-05 16:00:56.040000+00:00,True,2020-11-05 16:23:38.421836+00:00,0 days 00:22:42.381836
1039,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_302034,2020-11-05 18:10:53.857000+00:00,True,2020-11-05 18:14:33.960956+00:00,0 days 00:03:40.103956
1040,DESKTOP-GTJ7B1V:S-1-5-21-3137781443-2598908186...,t47E75B06-7B7B-4580-8152-B55559E1502C_1375164,2020-11-05 18:15:01.946000+00:00,False,2020-11-05 18:15:06.006260+00:00,0 days 00:00:04.060260


In [ ]:
# users_df.group_by('gcp_logon_id')[''].unique()
users_df['gcp_logon_id'].value_counts()

t73ED267D-10FE-4E7C-89E2-DF25824B5602_21861222    4
tE183CE41-B4BE-4477-98D7-60351D17AE42_208205      4
t0297182E-C0C5-42D1-8E07-2BD3C712903E_442591      4
t91FF8218-FF22-4BCA-A933-0AD4657F295B_527796      3
t1D10E273-1C22-41AD-BA42-D2B07C84550A_696525      3
                                                 ..
t3F159A9C-2A1F-4D59-A0CF-23D5DE40D01A_788417      1
t1754E6FC-BB79-4ADB-AD8F-FCC3DF3EC515_788417      1
t43992DD4-A9AF-40B8-B5BF-CF60484F400C_788417      1
t4BAF03F5-9BDB-4720-9E43-E77633F028D1_788417      1
tD9CB2EC2-5CF7-4433-B2FB-AB9DE8A0EBC1_292976      1
Name: gcp_logon_id, Length: 411, dtype: int64

We are interested in creating features that describe a user at a given timestamp. Please create a feature that describes how long the user has been logged in at a particular session.

#### 3. feature that describes how long the user has been logged in at a particular session

In [ ]:
users_df = users_df.groupby(['domain_sid', 'gcp_logon_id', 'logon_time', 'termination_time', 'session_terminated',])['session_duration'].sum()
users_df = users_df.reset_index()
users_df

,domain_sid,gcp_logon_id,logon_time,termination_time,session_terminated,session_duration
0,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_111207,2020-11-06 18:23:47.378000+00:00,2020-11-09 19:23:50.850351+00:00,False,3 days 01:00:03.472351
1,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_282735,2020-11-06 17:54:11.493000+00:00,2020-11-06 18:21:42.752718+00:00,True,0 days 00:27:31.259718
2,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_327233,2020-11-06 16:30:02.735000+00:00,2020-11-06 17:56:23.432259+00:00,False,0 days 01:26:20.697259
3,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t1D13E344-B3DD-4509-AE82-0D4A97BF55E7_377435,2020-11-06 14:27:19.853000+00:00,2020-11-06 14:34:36.806381+00:00,False,0 days 00:07:16.953381
4,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t252DB78E-1A3F-4BB7-B150-7900A28C2038_327746,2020-11-11 16:18:04.024000+00:00,2020-11-11 17:51:11.339920+00:00,False,0 days 03:06:14.631840
...,...,...,...,...,...,...
410,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE183CE41-B4BE-4477-98D7-60351D17AE42_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 05:02:03.921480+00:00,False,33 days 18:48:01.725920
411,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE5EBDBFE-6835-4AA1-A38F-67AE45889271_208205,2020-12-10 18:20:03.490000+00:00,2020-12-10 19:37:47.410098+00:00,False,0 days 01:17:43.920098
412,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tECD6130D-B254-4D79-BF44-1C93D7000650_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 00:44:48.017031+00:00,False,8 days 06:24:44.527031
413,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tF0123CB0-05F6-4745-AE4F-D6AD8AA1A9BE_208205,2020-12-10 18:20:03.490000+00:00,2020-12-18 23:05:01.846967+00:00,False,8 days 04:44:58.356967


We are interested in creating a feature that gives us the first and last timestamp for a day for a user on the network. This will give us the understanding of how the user working pattern is during a day over the past n days. 

In [ ]:
# for each day, please create a feature that provides the first timestamp of that date for that user

# df_clean['logon_date'] = df_clean['logon_time'].dt.date

# grouped_multiple = df_clean.groupby(['logon_date', 'domain_sid']).agg({'logon_time': ['min']})
# grouped_multiple.columns = ['first_time_stamp']
# grouped_multiple = grouped_multiple.reset_index()
# grouped_multiple

users_df['logon_date'] = users_df['logon_time'].dt.date

grouped_multiple = users_df.groupby(['logon_date', 'domain_sid']).agg({'logon_time': ['min', 'max']})
grouped_multiple.columns = ['first_time_stamp', 'last_time_stamp']
grouped_multiple = grouped_multiple.reset_index()
grouped_multiple.domain_sid[2]

'TAU:S-1-5-21-1075876963-3819156934-1349361978-1148'

In [ ]:
user_df = pd.merge(users_df, grouped_multiple,  how='left', left_on=['logon_date', 'domain_sid'], right_on = ['logon_date', 'domain_sid'])
user_df.head(5)

,domain_sid,gcp_logon_id,logon_time,termination_time,session_terminated,session_duration,logon_date,first_time_stamp,last_time_stamp
0,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_111207,2020-11-06 18:23:47.378000+00:00,2020-11-09 19:23:50.850351+00:00,False,3 days 01:00:03.472351,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00
1,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_282735,2020-11-06 17:54:11.493000+00:00,2020-11-06 18:21:42.752718+00:00,True,0 days 00:27:31.259718,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00
2,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_327233,2020-11-06 16:30:02.735000+00:00,2020-11-06 17:56:23.432259+00:00,False,0 days 01:26:20.697259,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00
3,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t1D13E344-B3DD-4509-AE82-0D4A97BF55E7_377435,2020-11-06 14:27:19.853000+00:00,2020-11-06 14:34:36.806381+00:00,False,0 days 00:07:16.953381,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00
4,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t252DB78E-1A3F-4BB7-B150-7900A28C2038_327746,2020-11-11 16:18:04.024000+00:00,2020-11-11 17:51:11.339920+00:00,False,0 days 03:06:14.631840,2020-11-11,2020-11-11 16:18:04.024000+00:00,2020-11-11 16:18:04.024000+00:00


In [ ]:
# create a boolean feature that describes if the timestamp is the first timestamp of the day

user_df['isFirstTimeStamp'] = user_df.apply(lambda row: row.logon_time == row.first_time_stamp, axis = 1)
user_df

,domain_sid,gcp_logon_id,logon_time,termination_time,session_terminated,session_duration,logon_date,first_time_stamp,last_time_stamp,isFirstTimeStamp
0,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_111207,2020-11-06 18:23:47.378000+00:00,2020-11-09 19:23:50.850351+00:00,False,3 days 01:00:03.472351,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False
1,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_282735,2020-11-06 17:54:11.493000+00:00,2020-11-06 18:21:42.752718+00:00,True,0 days 00:27:31.259718,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False
2,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_327233,2020-11-06 16:30:02.735000+00:00,2020-11-06 17:56:23.432259+00:00,False,0 days 01:26:20.697259,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False
3,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t1D13E344-B3DD-4509-AE82-0D4A97BF55E7_377435,2020-11-06 14:27:19.853000+00:00,2020-11-06 14:34:36.806381+00:00,False,0 days 00:07:16.953381,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,True
4,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t252DB78E-1A3F-4BB7-B150-7900A28C2038_327746,2020-11-11 16:18:04.024000+00:00,2020-11-11 17:51:11.339920+00:00,False,0 days 03:06:14.631840,2020-11-11,2020-11-11 16:18:04.024000+00:00,2020-11-11 16:18:04.024000+00:00,True
...,...,...,...,...,...,...,...,...,...,...
410,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE183CE41-B4BE-4477-98D7-60351D17AE42_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 05:02:03.921480+00:00,False,33 days 18:48:01.725920,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False
411,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE5EBDBFE-6835-4AA1-A38F-67AE45889271_208205,2020-12-10 18:20:03.490000+00:00,2020-12-10 19:37:47.410098+00:00,False,0 days 01:17:43.920098,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False
412,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tECD6130D-B254-4D79-BF44-1C93D7000650_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 00:44:48.017031+00:00,False,8 days 06:24:44.527031,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False
413,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tF0123CB0-05F6-4745-AE4F-D6AD8AA1A9BE_208205,2020-12-10 18:20:03.490000+00:00,2020-12-18 23:05:01.846967+00:00,False,8 days 04:44:58.356967,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False


In [ ]:
def function(current_timestamp, user_id):
  previous_day_fts = user_df.loc[(user_df['logon_date'] == (current_timestamp.date() - datetime.timedelta(days=1)))  & (user_df['domain_sid']==user_id)].first_time_stamp
  previous_day_lts = user_df.loc[(user_df['logon_date'] == (current_timestamp.date() - datetime.timedelta(days=1)))  & (user_df['domain_sid']==user_id)].last_time_stamp
    
  if previous_day_fts.empty:
    previous_day_fts = 'no record found'
  if previous_day_lts.empty:
    previous_day_lts = 'no record found'
  
  return previous_day_fts, previous_day_lts


  # print(df)
  # user = df_clean.loc[ user_df['logon_time']== current_timestamp, 'gcp_logon_id'].item()
  
  # current_date = df_clean.loc[ df_clean['timestamp']== current_timestamp, 'logon_date'].item()
  # previous_date = current_date - datetime.timedelta(days=1)
  # # print(previous_date)

  # group_df = df_clean.groupby(['logon_date', 'gcp_logon_id']).agg({'logon_time': ['min']})
  # group_df.columns = ['first_time_stamp']
  # group_df = group_df.reset_index()
  # # print(group_df)
  # row = group_df.loc[ (group_df['gcp_logon_id'] == user) & (group_df['logon_date'] == previous_date)]
  # if row.empty:
  #   return 'no record found'
  # else:
  #   print(user)
  #   print(current_date)
  #   return row['timestamp'].item() 

def function2(row):
  current_timestamp, user_id = row['logon_time'], row['domain_sid']
  return function(current_timestamp, user_id)

In [ ]:
# Using the function above create a feature that provides the first timestamp of the previous day for the current timestamp of the user
df3 = user_df.copy()
# df3['df3.apply(function2, axis=1))
print(df3.apply(function2, axis=1))
df3[['previous_day_fts']] = df3.apply(function2, axis=1,result_type="expand")[0]
# df3['previous_day_first_time_stamp'].value_counts()
df3

0                     (no record found, no record found)
1                     (no record found, no record found)
2                     (no record found, no record found)
3                     (no record found, no record found)
4      ([2020-11-10 17:30:27.903000+00:00], [2020-11-...
                             ...                        
410                   (no record found, no record found)
411                   (no record found, no record found)
412                   (no record found, no record found)
413                   (no record found, no record found)
414                   (no record found, no record found)
Length: 415, dtype: object


,domain_sid,gcp_logon_id,logon_time,termination_time,session_terminated,session_duration,logon_date,first_time_stamp,last_time_stamp,isFirstTimeStamp,previous_day_fts
0,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_111207,2020-11-06 18:23:47.378000+00:00,2020-11-09 19:23:50.850351+00:00,False,3 days 01:00:03.472351,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found
1,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_282735,2020-11-06 17:54:11.493000+00:00,2020-11-06 18:21:42.752718+00:00,True,0 days 00:27:31.259718,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found
2,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_327233,2020-11-06 16:30:02.735000+00:00,2020-11-06 17:56:23.432259+00:00,False,0 days 01:26:20.697259,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found
3,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t1D13E344-B3DD-4509-AE82-0D4A97BF55E7_377435,2020-11-06 14:27:19.853000+00:00,2020-11-06 14:34:36.806381+00:00,False,0 days 00:07:16.953381,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,True,no record found
4,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t252DB78E-1A3F-4BB7-B150-7900A28C2038_327746,2020-11-11 16:18:04.024000+00:00,2020-11-11 17:51:11.339920+00:00,False,0 days 03:06:14.631840,2020-11-11,2020-11-11 16:18:04.024000+00:00,2020-11-11 16:18:04.024000+00:00,True,14 2020-11-10 17:30:27.903000+00:00 Name: fi...
...,...,...,...,...,...,...,...,...,...,...,...
410,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE183CE41-B4BE-4477-98D7-60351D17AE42_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 05:02:03.921480+00:00,False,33 days 18:48:01.725920,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found
411,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE5EBDBFE-6835-4AA1-A38F-67AE45889271_208205,2020-12-10 18:20:03.490000+00:00,2020-12-10 19:37:47.410098+00:00,False,0 days 01:17:43.920098,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found
412,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tECD6130D-B254-4D79-BF44-1C93D7000650_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 00:44:48.017031+00:00,False,8 days 06:24:44.527031,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found
413,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tF0123CB0-05F6-4745-AE4F-D6AD8AA1A9BE_208205,2020-12-10 18:20:03.490000+00:00,2020-12-18 23:05:01.846967+00:00,False,8 days 04:44:58.356967,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found


In [ ]:
# Using the function above create a feature that provides the last timestamp of the previous day for the user (be creative)

df3[['previous_day_fts', 'previous_day_lts']] = df3.apply(function2, axis=1,result_type="expand")
df3


,domain_sid,gcp_logon_id,logon_time,termination_time,session_terminated,session_duration,logon_date,first_time_stamp,last_time_stamp,isFirstTimeStamp,previous_day_fts,previous_day_lts
0,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_111207,2020-11-06 18:23:47.378000+00:00,2020-11-09 19:23:50.850351+00:00,False,3 days 01:00:03.472351,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found,no record found
1,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_282735,2020-11-06 17:54:11.493000+00:00,2020-11-06 18:21:42.752718+00:00,True,0 days 00:27:31.259718,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found,no record found
2,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_327233,2020-11-06 16:30:02.735000+00:00,2020-11-06 17:56:23.432259+00:00,False,0 days 01:26:20.697259,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found,no record found
3,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t1D13E344-B3DD-4509-AE82-0D4A97BF55E7_377435,2020-11-06 14:27:19.853000+00:00,2020-11-06 14:34:36.806381+00:00,False,0 days 00:07:16.953381,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,True,no record found,no record found
4,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t252DB78E-1A3F-4BB7-B150-7900A28C2038_327746,2020-11-11 16:18:04.024000+00:00,2020-11-11 17:51:11.339920+00:00,False,0 days 03:06:14.631840,2020-11-11,2020-11-11 16:18:04.024000+00:00,2020-11-11 16:18:04.024000+00:00,True,14 2020-11-10 17:30:27.903000+00:00 Name: fi...,14 2020-11-10 17:30:27.903000+00:00 Name: la...
...,...,...,...,...,...,...,...,...,...,...,...,...
410,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE183CE41-B4BE-4477-98D7-60351D17AE42_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 05:02:03.921480+00:00,False,33 days 18:48:01.725920,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found
411,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE5EBDBFE-6835-4AA1-A38F-67AE45889271_208205,2020-12-10 18:20:03.490000+00:00,2020-12-10 19:37:47.410098+00:00,False,0 days 01:17:43.920098,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found
412,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tECD6130D-B254-4D79-BF44-1C93D7000650_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 00:44:48.017031+00:00,False,8 days 06:24:44.527031,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found
413,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tF0123CB0-05F6-4745-AE4F-D6AD8AA1A9BE_208205,2020-12-10 18:20:03.490000+00:00,2020-12-18 23:05:01.846967+00:00,False,8 days 04:44:58.356967,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found


In [ ]:
# create a function that gives the average value of the first timestamp (exclude the date) of a day over the past n_days 





In [ ]:
# Using the function above, create a feature that gives the average value of the first timestamp of a day over the past 7 days





In [ ]:
# show the final table

final_df = df3.copy()
final_df

,domain_sid,gcp_logon_id,logon_time,termination_time,session_terminated,session_duration,logon_date,first_time_stamp,last_time_stamp,isFirstTimeStamp,previous_day_fts,previous_day_lts
0,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_111207,2020-11-06 18:23:47.378000+00:00,2020-11-09 19:23:50.850351+00:00,False,3 days 01:00:03.472351,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found,no record found
1,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_282735,2020-11-06 17:54:11.493000+00:00,2020-11-06 18:21:42.752718+00:00,True,0 days 00:27:31.259718,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found,no record found
2,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t025D5E57-D08E-4E0B-8C6D-7B522021E9E9_327233,2020-11-06 16:30:02.735000+00:00,2020-11-06 17:56:23.432259+00:00,False,0 days 01:26:20.697259,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,False,no record found,no record found
3,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t1D13E344-B3DD-4509-AE82-0D4A97BF55E7_377435,2020-11-06 14:27:19.853000+00:00,2020-11-06 14:34:36.806381+00:00,False,0 days 00:07:16.953381,2020-11-06,2020-11-06 14:27:19.853000+00:00,2020-11-06 18:23:47.378000+00:00,True,no record found,no record found
4,DESKTOP-5PB3K54:S-1-5-21-1496041904-3633278568...,t252DB78E-1A3F-4BB7-B150-7900A28C2038_327746,2020-11-11 16:18:04.024000+00:00,2020-11-11 17:51:11.339920+00:00,False,0 days 03:06:14.631840,2020-11-11,2020-11-11 16:18:04.024000+00:00,2020-11-11 16:18:04.024000+00:00,True,14 2020-11-10 17:30:27.903000+00:00 Name: fi...,14 2020-11-10 17:30:27.903000+00:00 Name: la...
...,...,...,...,...,...,...,...,...,...,...,...,...
410,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE183CE41-B4BE-4477-98D7-60351D17AE42_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 05:02:03.921480+00:00,False,33 days 18:48:01.725920,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found
411,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tE5EBDBFE-6835-4AA1-A38F-67AE45889271_208205,2020-12-10 18:20:03.490000+00:00,2020-12-10 19:37:47.410098+00:00,False,0 days 01:17:43.920098,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found
412,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tECD6130D-B254-4D79-BF44-1C93D7000650_208205,2020-12-10 18:20:03.490000+00:00,2020-12-19 00:44:48.017031+00:00,False,8 days 06:24:44.527031,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found
413,WINDEV2011EVAL:S-1-5-21-2233568566-729489526-2...,tF0123CB0-05F6-4745-AE4F-D6AD8AA1A9BE_208205,2020-12-10 18:20:03.490000+00:00,2020-12-18 23:05:01.846967+00:00,False,8 days 04:44:58.356967,2020-12-10,2020-12-10 15:46:59.939000+00:00,2020-12-10 18:20:03.490000+00:00,False,no record found,no record found


## Step 4 - Visualization

We are interested in creating some visualizations around the user behavior. Please create two figures using the features that you have created. Please make them descriptive and clear.

In [ ]:
# figure 1 - illustrate an interesting finding about a particular user of your choosing





In [ ]:
# figure 2 - comparison of two users of your choosing




